# Ex10 - Texturas

Nesta atividade, vocês vão exercitar alguns dos conceitos de textura. Para evitar erros de execução, utilize apenas uma célula de código para cada parte desta atividade.

### Parte 1 - Skybox

Skybox é uma técnica de criação de background utilizada em jogos para simular o horizonte que rodeia o jogador. Resumidamente, esta técnica consiste em criar um cubo ao redor da cena criada. Nesse cubo, as faces internas são texturizadas com as imagens da paisagem que se deseja simular. Utilize uma das texturas presentes na pasta ./cg/images/textures/ para criar um skybox simples em torno de uma câmera posicionada no ponto (0,0,0). Além disso, utilize as teclas de seta para rotacionar o ponto de visão da câmera a fim de permitir visualizar o resultado da renderização do cubo. As imagens abaixo ilustram o efeito esperado ao rotacionar o ponto de visão da câmera. Os notebooks ([37_Renderizando_uma_textura_por_completo](37_Renderizando_uma_textura_por_completo.ipynb)) e ([38_Renderizando_parte_de_uma_textura](38_Renderizando_parte_de_uma_textura.ipynb)) exemplificam como utilizar texturas em renderizações. 

<table>
    <tr>
        <td> <img src='cg/images/ex10_image_1.png' style="width:400px"> </td>
        <td> <img src='cg/images/ex10_image_2.png' style="width:400px"></td>
        <td> <img src='cg/images/ex10_image_3.png' style="width:400px"></td>
    </tr>
</table>

In [1]:
import glm
import time
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SquareMesh_v3 import SquareMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 0.0)
        self.cameraFront = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)

        vertex_position = np.array([
            -1.0, -1.0, 0.0, 1.0, # Triângulo 1
             1.0, -1.0, 0.0, 1.0,
            -1.0,  1.0, 0.0, 1.0,
             1.0, -1.0, 0.0, 1.0, # Triângulo 2
             1.0,  1.0, 0.0, 1.0,
            -1.0,  1.0, 0.0, 1.0],
            dtype=np.float32)
        
        self.vertex_tex = np.array([
             0.00,  1.00, # Triângulo 1
             1.00,  1.00,
             0.00,  0.00,
             1.00,  1.00, # Triângulo 2
             1.00,  0.00,
             0.00,  0.00],
            dtype=np.float32)
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        # as texturas carregadas possuem frames com o mesmo tamanho
        frame_size = np.array([1.0 / 4, 1.0 / 3], dtype=np.float32)
        self.tex_first_frame = self.vertex_tex.reshape((6, 2)) * frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 5)
        y = np.linspace(0, 1, 4)
        self.tex_offset_x, self.tex_offset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.tex_offset_x = self.tex_offset_x.reshape((12)).astype(np.float32)
        self.tex_offset_y = self.tex_offset_y.reshape((12)).astype(np.float32)
        
        #carrega textura do skybox
        self.texture_1 = Texture('./cg/images/textures/skybox2.png')
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(vertex_position, vertex_tex=self.vertex_tex)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        tex_coord_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexTextureCoordLoc(tex_coord_loc)
        
        # configura a cor de background
        gl.glClearColor(0.9, 0.9, 0.9, 1)
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
    def paintGL(self):
            
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza cena
        self.renderSkybox()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def renderSkybox(self):
        
        # configura/ativa a textura a ser usada
        self.shaderProgram.bindTexture2D(self.texture_1.getTextureID())
        
        self.renderSquare(glm.radians(0), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[5], self.tex_offset_y[5]) # front
        self.renderSquare(glm.radians(90), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[4], self.tex_offset_y[4]) # left
        self.renderSquare(glm.radians(-90), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[6], self.tex_offset_y[6]) # right
        self.renderSquare(glm.radians(90), glm.vec3(1.0, 0.0, 0.0), self.tex_offset_x[1], self.tex_offset_y[1]) # top
        self.renderSquare(glm.radians(-90), glm.vec3(1.0, 0.0, 0.0), self.tex_offset_x[9], self.tex_offset_y[9]) # bottom
        self.renderSquare(glm.radians(180), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[7], self.tex_offset_y[7]) # back
        
        # desativa a textura
        self.shaderProgram.releaseTexture2D()
        
    def renderSquare(self, rot_angle, rot_axis, tex_offset_x, tex_offset_y):
        
        # cria matriz de transformação do quadrado do skybox
        rot = glm.rotate(glm.mat4(), rot_angle, rot_axis)        
        model_matrix = glm.translate(rot, glm.vec3(0.0, 0.0, -0.99))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # calcula as coordenadas de texturas e atualiza as coordenadas da GPU
        frame_tex = self.tex_first_frame + np.array([tex_offset_x, tex_offset_y])
        self.squareRenderer.updateVertexTextureCoord(frame_tex)

        # renderiza o quadradado com as novas coordenadas de textura
        self.squareRenderer.render()
        
    def resizeGL(self, width, height):
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        step = 0.5
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            rot_axi = glm.cross(self.cameraFront, self.cameraUp)
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), rot_axi)) * self.cameraFront
            self.cameraUp = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), rot_axi)) * self.cameraUp
            
        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            rot_axi = glm.cross(self.cameraFront, self.cameraUp)
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), rot_axi)) * self.cameraFront
            self.cameraUp = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), rot_axi)) * self.cameraUp
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), self.cameraUp)) * self.cameraFront
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), self.cameraUp)) * self.cameraFront
        
        # atualiza a câmera
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(900, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

trying to open ./cg/images/textures/skybox2.png
opened file: size= (1024, 768) format= PNG mode= RGBA


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Parte 2 - Objetos texturizados

Adicione à renderização da Parte 1 pelo menos dois objetos texturizados ([41_Objetos_com_texturas_e_height_maps](41_Objetos_com_texturas_e_height_maps.ipynb) e [40_Objetos_com_texturas](40_Objetos_com_texturas.ipynb)) e uma fonte de luz direcional ([34_Terreno_com_esferas_iluminacao_fonte_de_luz_direcional](34_Terreno_com_esferas_iluminacao_fonte_de_luz_direcional.ipynb)) para simular o sol. A pasta ./cg/images/textures/ possui algumas texturas que podem ser utilizadas nos objetos.

In [2]:
import glm
import time
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.shader_programs.PhongShadingShaderProgram_v2 import PhongShadingShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SquareMesh_v3 import SquareMesh
from cg.models.SphereMesh_v3 import SphereMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 0.0)
        self.cameraFront = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)

        vertex_position = np.array([
            -1.0, -1.0, 0.0, 1.0, # Triângulo 1
             1.0, -1.0, 0.0, 1.0,
            -1.0,  1.0, 0.0, 1.0,
             1.0, -1.0, 0.0, 1.0, # Triângulo 2
             1.0,  1.0, 0.0, 1.0,
            -1.0,  1.0, 0.0, 1.0],
            dtype=np.float32)
        
        self.vertex_tex = np.array([
             0.00,  1.00, # Triângulo 1
             1.00,  1.00,
             0.00,  0.00,
             1.00,  1.00, # Triângulo 2
             1.00,  0.00,
             0.00,  0.00],
            dtype=np.float32)
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        # as texturas carregadas possuem frames com o mesmo tamanho
        frame_size = np.array([1.0 / 4, 1.0 / 3], dtype=np.float32)
        self.tex_first_frame = self.vertex_tex.reshape((6, 2)) * frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 5)
        y = np.linspace(0, 1, 4)
        self.tex_offset_x, self.tex_offset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.tex_offset_x = self.tex_offset_x.reshape((12)).astype(np.float32)
        self.tex_offset_y = self.tex_offset_y.reshape((12)).astype(np.float32)
        
        #carrega textura do skybox
        self.texture_1 = Texture('./cg/images/textures/skybox2.png')
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(vertex_position, vertex_tex=self.vertex_tex)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        tex_coord_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexTextureCoordLoc(tex_coord_loc)
        
        # configura a cor de background
        gl.glClearColor(0.9, 0.9, 0.9, 1)
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # cria malha esférica
        sphere_mesh = SphereMesh(0.4, 50, 50)
        
        self.sphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                                 vertex_indices=sphere_mesh.getVertexIndices(),
                                                 vertex_normal=sphere_mesh.getVertexNormals(),
                                                 vertex_tex=sphere_mesh.getVertexTextureCoord())
        
        # cria um shader de iluminação Phong
        self.phongShaderProgram = PhongShadingShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.phongShaderProgram.getVertexPositionLoc()
        normal_loc = self.phongShaderProgram.getVertexNormalLoc()
        tex_coord_loc = self.phongShaderProgram.getVertexTextureCoordLoc()
        
        self.sphereRenderer.setVertexPositionLoc(position_loc)
        self.sphereRenderer.setVertexNormalLoc(normal_loc)
        self.sphereRenderer.setVertexTextureCoordLoc(tex_coord_loc)
        
        # ativa o shader programa para configurar as propriedades da luz
        self.phongShaderProgram.bind()
        
        # permite usar uma cor única para todos os vértices
        # tem o mesmo funcionamento da função useUniformColor do SimpleShader
        self.phongShaderProgram.useUniformMaterialColor(True)
        
        #configura as cores da luz
        self.phongShaderProgram.setUniformLightAmbient(np.array([0.2, 0.2, 0.2], dtype=np.float32))
        self.phongShaderProgram.setUniformLightIntensity(np.array([1.0, 1.0, 1.0], dtype=np.float32))
        
        #altera a tipo de luz para fonte de luz direcional (default: luz pontual)
        self.phongShaderProgram.setUniformLightMode(PhongShadingShaderProgram.DIRECTIONAL_LIGHT)
        
        #configurar a direção da luz
        #tem que ser transformada como qualquer outro objeto
        #No entanto, por ser um vetor, não sofre translação
        #Então, é transformada apenas pela sub-matrix 3x3 da matriz model-view
        light_direction = glm.vec3(0.0, -1.0, 0.0)
        self.phongShaderProgram.setUniformLightDirection(glm.mat3(self.viewMatrix) * light_direction)
        
        self.phongShaderProgram.release()
        
        # carrega as texturas
        self.texture1 = Texture('./cg/images/textures/Pebbles_021_SD/Pebbles_021_4K_basecolor.jpg')
        self.texture2 = Texture('./cg/images/textures/Lava_005_SD/Lava_005_COLOR.jpg')
        self.texture3 = Texture('./cg/images/textures/Brick_Wall_018_SD/Brick_Wall_018_basecolor.jpg')
        self.texture4 = Texture('./cg/images/textures/Incrusted_Gems_001_SD/Incrusted_Gems_001_COLOR.jpg')
        
    def paintGL(self):
            
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza cena
        self.renderSkybox()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # ativa o shader program que será executado pela GPU
        self.phongShaderProgram.bind()
        
        # renderiza cena
        self.renderSpheres()
        
        # desativa o shader program
        self.phongShaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def renderSkybox(self):
        
        # configura/ativa a textura a ser usada
        self.shaderProgram.bindTexture2D(self.texture_1.getTextureID())
        
        self.renderSquare(glm.radians(0), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[5], self.tex_offset_y[5]) # front
        self.renderSquare(glm.radians(90), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[4], self.tex_offset_y[4]) # left
        self.renderSquare(glm.radians(-90), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[6], self.tex_offset_y[6]) # right
        self.renderSquare(glm.radians(90), glm.vec3(1.0, 0.0, 0.0), self.tex_offset_x[1], self.tex_offset_y[1]) # top
        self.renderSquare(glm.radians(-90), glm.vec3(1.0, 0.0, 0.0), self.tex_offset_x[9], self.tex_offset_y[9]) # bottom
        self.renderSquare(glm.radians(180), glm.vec3(0.0, 1.0, 0.0), self.tex_offset_x[7], self.tex_offset_y[7]) # back
        
        # desativa a textura
        self.shaderProgram.releaseTexture2D()
        
    def renderSquare(self, rot_angle, rot_axis, tex_offset_x, tex_offset_y):
        
        # cria matriz de transformação do quadrado do skybox
        rot = glm.rotate(glm.mat4(), rot_angle, rot_axis)        
        model_matrix = glm.translate(rot, glm.vec3(0.0, 0.0, -0.99))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # calcula as coordenadas de texturas e atualiza as coordenadas da GPU
        frame_tex = self.tex_first_frame + np.array([tex_offset_x, tex_offset_y])
        self.squareRenderer.updateVertexTextureCoord(frame_tex)

        # renderiza os quadradado com as novas coordenadas de textura
        self.squareRenderer.render()
    
    def renderSpheres(self):
        
        scale = glm.scale(glm.mat4(), glm.vec3(0.5, 0.5, 0.5))
        
        #calcula a matriz model-view da esfera do fundo
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, 1.1))
        mv_matrix = self.viewMatrix * transl * scale
        self.renderSphere(mv_matrix, self.texture1)
        
        #calcula a matriz model-view da esfera da direita
        transl = glm.translate(glm.mat4(), glm.vec3(1.1, 0.0, 0.0))
        mv_matrix = self.viewMatrix * transl * scale
        self.renderSphere(mv_matrix, self.texture2)
        
        #calcula a matriz model-view da esfera da esquerda
        transl = glm.translate(glm.mat4(), glm.vec3(-1.1, 0.0, 0.0))
        mv_matrix = self.viewMatrix * transl * scale
        self.renderSphere(mv_matrix, self.texture3)
        
        #calcula a matriz model-view da esfera da frente
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -1.1))
        mv_matrix = self.viewMatrix * transl * scale
        self.renderSphere(mv_matrix, self.texture4)
    
    #renderiza uma esfera
    def renderSphere(self, mv_matrix, texture):
        
        #configura a matriz mvp para renderizar a cena
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * mv_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(mv_matrix)
        
        # configura/ativa a textura a ser usada 
        self.phongShaderProgram.bindTexture2D(texture.getTextureID())
        
        self.sphereRenderer.render()
        
        # desativa a textura
        self.phongShaderProgram.releaseTexture2D()
        
    def resizeGL(self, width, height):
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        step = 0.5
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            rot_axi = glm.cross(self.cameraFront, self.cameraUp)
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), rot_axi)) * self.cameraFront
            self.cameraUp = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), rot_axi)) * self.cameraUp
            
        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            rot_axi = glm.cross(self.cameraFront, self.cameraUp)
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), rot_axi)) * self.cameraFront
            self.cameraUp = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), rot_axi)) * self.cameraUp
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(1), self.cameraUp)) * self.cameraFront
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraFront = glm.mat3(glm.rotate(glm.mat4(), glm.radians(-1), self.cameraUp)) * self.cameraFront
        
        # atualiza a câmera
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(900, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

trying to open ./cg/images/textures/skybox2.png
opened file: size= (1024, 768) format= PNG mode= RGBA
trying to open ./cg/images/textures/Pebbles_021_SD/Pebbles_021_4K_basecolor.jpg
opened file: size= (1024, 1024) format= JPEG mode= RGB
trying to open ./cg/images/textures/Lava_005_SD/Lava_005_COLOR.jpg
opened file: size= (1024, 1024) format= JPEG mode= RGB
trying to open ./cg/images/textures/Brick_Wall_018_SD/Brick_Wall_018_basecolor.jpg
opened file: size= (1024, 1024) format= JPEG mode= RGB
trying to open ./cg/images/textures/Incrusted_Gems_001_SD/Incrusted_Gems_001_COLOR.jpg
opened file: size= (1024, 1024) format= JPEG mode= RGB


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Parte 3 - Sprites animados

Em alguns games 2D, a animação de sprites é utilizada no lugar de objetos em movimento compostos por malha de triângulos. O notebook [39_Renderizando_sprites_animados](39_Renderizando_sprites_animados.ipynb) exemplifica essa técnica. Utilize uma das texturas de sprites presentes na pasta ./cg/images/textures/ para montar pelo menos duas animações que são disparadas por eventos de teclado. Por exemplo, se a tecla de seta 'para direita' for pressionada, a animação do sprite andando para direita começa.